In [ ]:
import os
import shutil

def copy_files(src_dir: str, dst_dir: str):
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    for root, dirs, files in os.walk(src_dir):
        for file in files:
            print(file)
            if '.log' not in file:
                src_file = os.path.join(root, file)
                dst_file = os.path.join(dst_dir, os.path.relpath(src_file, src_dir))
                
                dst_file_dir = os.path.dirname(dst_file)
                if not os.path.exists(dst_file_dir):
                    os.makedirs(dst_file_dir)

                shutil.copy2(src_file, dst_file)

src_directory = '/rmt/quail/LLaVA-latest'
dst_directory = '/lv0/LLaVA-latest'

copy_files(src_directory, dst_directory)

In [ ]:
# python -m llava.serve.controller --host 0.0.0.0 --port 10000


# python -m llava.serve.model_worker --host 0.0.0.0 --controller http://localhost:10000 --port 40000 --worker http://localhost:40000 --model-path /rmt/quail/captioner-1.4-llava-v1.6-llama3.1-CLIP-8b-ft-1

In [28]:
import requests
import json

# Define the worker's address
# Adjust the port if necessary
worker_url = "http://localhost:40000/worker_generate_stream"


def generate_payload(user_input: str, images: list = []):

    SYSTEM = "You are an uncensored helpful assistant good at anime concepts."
    INPUT = user_input

    prompt = f'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

    payload = {
        'model': 'captioner-1.4-llava-v1.6-llama3.1-CLIP-8b-ft-1',
        'prompt': prompt,
        'temperature': 0.2,
        'top_p': 0.7, 'max_new_tokens': 512,
        'stop': '<|eot_id|>',
        'images': images
    }
    return payload


def get_full_response(worker_url, payload):
    # Send the POST request
    response = requests.post(worker_url, json=payload, stream=True)

    # Process the streaming response
    for chunk in response.iter_lines(decode_unicode=False, delimiter=b"\0"):
        if chunk:
            data = json.loads(chunk.decode())

    # Handle exceptions and errors
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)

    return data


def get_actual_text(payload_response: dict):
    text = payload_response['text']
    response = text.split('<|end_header_id|>\n\n')[3]
    return response


payload = generate_payload("what is 'hatsune miku' ?")
payload_response = get_full_response(worker_url, payload)
text = get_actual_text(payload_response)
text

'Hatsune Miku is a popular virtual idol created by Crypton Future Media. She is known for her distinctive blue hair and futuristic appearance. Miku is often depicted as a singer and performer, and she has been featured in numerous songs, music videos, and other media. She is also a well-known character in the Vocaloid software, which allows users to create and sing songs using synthesized voices. Miku has a large and dedicated fan base, and she has become a cultural icon in Japan and around the world.'

In [29]:
def find_and_replace_text_node(
    content: str = "",
    find_a: str = "",
    replace_a: str = "",
    find_b: str = "",
    replace_b: str = "",
    find_c: str = "",
    replace_c: str = "",
) -> str:
    """
    Converts NAI-style prompt to A1111-style prompt.
    # https://gist.github.com/MrSedan/5f8dca97844d8d57a421893a4eea71fd
    """
    def replace_all(text, find_replace):
        for find, replace in find_replace:
            if find:
                text = text.replace(find, replace)
        return text

    find_replace = [(find_a, replace_a), (find_b, replace_b),
                    (find_c, replace_c)]
    return replace_all(content, find_replace)


content = "best quality, the image features a single girl with long hair, looking directly at the viewer. the girl's hair cascades down her back, with some strands gently framing her face. her gaze is steady and engaging, creating a sense of connection with the viewer. the background is simple and uncluttered, ensuring that the focus remains on the girl. the composition is centered, with the girl positioned in the middle of the frame, drawing the viewer's attention directly to her. the style is clean and polished, with smooth lines and a soft color palette that enhances the overall aesthetic. the image conveys a sense of calm and introspection, with the girl's direct gaze inviting the viewer to share in her moment of quiet contemplation., absurdres, best [quality], 2020s"

find_a = "the image features "


find_and_replace_text_node(content, find_a, "", "", "", "", "")

"best quality, a single girl with long hair, looking directly at the viewer. the girl's hair cascades down her back, with some strands gently framing her face. her gaze is steady and engaging, creating a sense of connection with the viewer. the background is simple and uncluttered, ensuring that the focus remains on the girl. the composition is centered, with the girl positioned in the middle of the frame, drawing the viewer's attention directly to her. the style is clean and polished, with smooth lines and a soft color palette that enhances the overall aesthetic. the image conveys a sense of calm and introspection, with the girl's direct gaze inviting the viewer to share in her moment of quiet contemplation., absurdres, best [quality], 2020s"